# Data acquisition from .pdf files

This notebook acquires data from the dataset2/falencias

Requirements:

*   textract - Enables easier extraction of text from .pdf files
*   pandas - Provides dataframes for storage, and conversion to .csv

The next cells are needed to run the notebook inside the Google Colaboratory platform, with the datasets in Google Drive

In [ ]:
!pip install textract

Setting up connection to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# dataset2 location in Google Drive
PATH_falencias = '/content/drive/Shared drives/NLP Jurídico - Lab. Ciência de Dados/Dados PDFs falencias'
PATH_save = '/content/drive/Shared drives/NLP Jurídico - Lab. Ciência de Dados'

The next cell is needed to run the notebook locally, with the datasets available in a local disk

In [ ]:
PATH_falencias = '/run/media/raktanaka/4387-FFCB/falenciaspdf'
PATH_save = '/run/media/raktanaka/4387-FFCB'

In [ ]:
import os
import pandas as pd
import textract
import unicodedata
import re

In [ ]:
df = pd.DataFrame(columns=['n_processo', 'tipo_documento', 'string', 'data_doc', 'assinado_por', 'n_folha_inicio', 'n_folha_fim'])

log_succ = open(os.path.join(PATH_save, 'pdf_success.log'), 'w')
log_fail = open(os.path.join(PATH_save, 'pdf_error.log'), 'w')

processos = os.listdir(PATH_falencias)
processos.sort()
tot_processos = len(processos)
p_counter = 1

for processo in processos: # For each action
    print('PROCESSO ' + str(p_counter) + '\\' + str(tot_processos))
    p_counter += 1
    PATH_processo = os.path.join(PATH_falencias, processo)
    # Needed to remove hidden files/folders in MacOS or Windows
    if not processo.startswith('.') or not processo.startswith('System'):
        arquivos = [f for f in os.listdir(PATH_processo) if os.path.isfile(os.path.join(PATH_processo, f))]
        pastas = [f for f in os.listdir(PATH_processo) if not os.path.isfile(os.path.join(PATH_processo, f))]

        arquivos.sort()
        pastas.sort()
        a_counter = 1
        tot_arquivos = len(arquivos)
        for arquivo in arquivos:
            print('  ARQUIVO ' + str(a_counter) + '\\' + str(tot_arquivos))
            a_counter += 1
            try:
                index_ini_parent = arquivo.find('(')
                tipo_documento = arquivo[:index_ini_parent-1]
                # Normalizes graphic accentuation 
                tipo_documento = unicodedata.normalize('NFC', tipo_documento)
                tipo_documento = tipo_documento.lower()
                n_folha_inicio = 0
                n_folha_fim = 0
                nome_assinador = ''

                pag_encontrada = False
                nome_do_arquivo = arquivo[index_ini_parent:]

                '''
                Gets the initial and final page from the file name
                '''
                # Tries to get what's between two parenthesis (...). If the string "pag"
                # isn't present, tries to get two more parenthesis (...) in the file name. 
                # This is crucial because there are filenames like "Bla (AR) (pag. xxx - xxx).pdf"

                while(not pag_encontrada):
                    index_ini_parent = nome_do_arquivo.find('(')
                    index_fim_parent = nome_do_arquivo.find(')')
                    
                    # Checks if there're still substrings with "(...)"
                    if index_ini_parent != -1:
                        # Checks if there's something like "(pag)" within the substring
                        if 'pag' in nome_do_arquivo[index_ini_parent:index_fim_parent]: 
                            pag_encontrada = True
                            # Checks if there're one or two pages in the file name
                            index_traco = nome_do_arquivo.find('-')
                            if index_traco != -1: 
                                n_folha_inicio = int(nome_do_arquivo[index_ini_parent+4:index_traco-1])
                                n_folha_fim = int(nome_do_arquivo[index_traco+1:index_fim_parent])
                            else: 
                                n_folha_inicio = int(nome_do_arquivo[index_ini_parent+4:index_fim_parent])
                                n_folha_fim = int(nome_do_arquivo[index_ini_parent+4:index_fim_parent])
                    else: 
                        pag_encontrada = True
                    nome_do_arquivo = nome_do_arquivo[index_fim_parent+1:]

                '''
                Extracts the string from the PDF
                '''
                if (len(arquivo) > 4 and arquivo[-4:] == '.pdf'):
                    text = textract.process(PATH_processo + '/' + arquivo)
                    text = text.decode("utf-8")
                    text = unicodedata.normalize('NFC', text)
                    text = text.lower()
                    text = text.replace('\n', '')
                    data_doc = ''

                    '''
                    Tries to find the assignature date 
                    '''
                    index_date = text.find("protocolado em")
                    if index_date != -1:
                        data_doc = text[index_date+15:index_date+25]
                    else: 
                        index_date = text.find("liberado nos autos em")
                        if index_date != -1: 
                            data_doc = text[index_date+22:index_date+32]
                    '''
                    Tries to find who has signed the documento
                    '''
                    index_name = text.find("assinado digitalmente por")
                    if index_name != -1:
                        index_end_name = text[index_name:].find(',')
                        nome_assinador = text[index_name+26:index_name+index_end_name]

                    # Filters the PDF string, removing the stopwords and other garbage. 
                    text = re.sub('(para conferir o original)(.+?)(e código [0-9A-za-z]+\.)', '', text)
                    text = re.sub('(este documento é cópia do original).+?(às [0-9]{2}:[0-9]{2} , sob o número [A-Za-z0-9]+\.)', '', text)
                    text = re.sub('(este documento é cópia do original).+?(liberado nos autos em [0-9]{2}/[0-9]{2}/[0-9]{4} às [0-9]{2}:[0-9]{2} . )', '', text)

                df = df.append({'n_processo': processo, 'tipo_documento': tipo_documento,
                                'string': text, 'data_doc': data_doc, 'assinado_por': nome_assinador,'n_folha_inicio': n_folha_inicio, 
                                'n_folha_fim':n_folha_fim}, ignore_index=True)
                #print(arquivo)
                #print(processo, tipo_documento, data_doc, n_folha_inicio, n_folha_fim)
                log_succ.write(processo + ' ' + arquivo + '\n')
            
            except: 
                #print('Ocorreu um problema ao processar o seguinte documento: ')
                #print(arquivo)
                log_fail.write(processo + ' ' + arquivo + '\n')
                pass
    
log_succ.close()
log_fail.close()

In [ ]:
# Stores the DataFrame in a csv
csv_name = 'dadospdf.csv'
csv_file = os.path.join(PATH_save, csv_name)
save = open(csv_file, 'w')
df.to_csv(save)
save.close()